# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1

In [2]:
!pip install torch torchvision

---

In [3]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [ ]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [26]:
# from glob import glob

# text_lines = []

# for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
#     with open(file_path, "r") as file:
#         file_text = file.read()

#     text_lines += file_text.split("# ")

    # print(text_lines)
    # print('\n')
    # print(len(text_lines))
    # print('\n\n\n\n')

# DeepSeek Prompt --
# 请用 python 写一段 code，读取当前目录下一个 markdown 文件 mfd.md， 读取的内容按 "\n#" 或者 "\n**" 分割为独立字符串，
# 并删除字符串中的 “#" 标题符或者 ”*"加粗符，去除空行，并将各非空字符串填入空列表 text_lines 中


import re
import os

def process_markdown_file(file_path):
    text_lines = []
    
    if not os.path.exists(file_path):
        print(f"文件 {file_path} 不存在")
        return text_lines
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            
            # 按 \n# 或 \n** 分割内容
            segments = re.split(r'\n#|\n\*\*', content)
            
            for segment in segments:
                # 删除可能残留的行首#和**（来自分割）
                segment = re.sub(r'^#+\s*', '', segment)  # 删除行首标题标记
                segment = segment.replace('**', '')       # 删除加粗标记
                
                # 去除首尾空白字符
                segment = segment.strip()
                
                # 跳过空字符串
                if segment:
                    text_lines.append(segment)
                    
    except Exception as e:
        print(f"读取文件时出错: {e}")
    
    return text_lines

# 使用示例
text_lines = process_markdown_file('mfd.md')

# 显示结果
print(f"处理后的段数: {len(text_lines)}")
for i, segment in enumerate(text_lines, 1):
    lines = segment.split('\n')
    print(f"--- 段 {i} ({len(lines)} 行) ---")
    for j, line in enumerate(lines, 1):
        print(f"{j:3}: {line}")
    print()

处理后的段数: 416
--- 段 1 (1 行) ---
  1: 中华人民共和国民法典

--- 段 2 (1 行) ---
  1: （二）物权编

--- 段 3 (1 行) ---
  1: 第一章 一般规定

--- 段 4 (1 行) ---
  1: 第二百零四条 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。

--- 段 5 (1 行) ---
  1: 第二百零五条 本编调整因物的归属和利用产生的民事关系。

--- 段 6 (3 行) ---
  1: 第二百零六条 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。
  2: 国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。
  3: 国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。

--- 段 7 (1 行) ---
  1: 第二百零七条 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。

--- 段 8 (1 行) ---
  1: 第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。

--- 段 9 (2 行) ---
  1: 第二百零九条 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。
  2: 依法属于国家所有的自然资源，所有权可以不登记。

--- 段 10 (2 行) ---
  1: 第二百一十条 不动产登记，由不动产所在地的登记机构办理。
  2: 国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。

--- 段 11 (2 行) ---
  1: 第二百一十一条 当事人申请登记，应当根据不同登记事项提供材料。
  2: 申请登记材料以及登记事项相关信息，可以公开查询。

--- 段 12 (6 行) ---
  1: 第二百一十二条 登记机构应当履行下列职责：
  2: （一）审查申请人提供的材料；
  3: （二）询问申请人；
  4: （三）如实、及时登记；
  5: （四）法律、行政法规规定的其他职责。
  6: 申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予

In [27]:
len(text_lines)

416

In [29]:
# print(text_lines)

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [30]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [31]:
# from pymilvus import model as milvus_model

# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-pJp5Zk42P34zeLRB3b55E9EeAe87442e99E4923275512552', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

生成一个**测试向量**并打印其维度和前几个元素。

In [32]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])
# print(test_embedding[:200])

512
[-0.02814663  0.00428726 -0.01852599  0.08190062 -0.03156214 -0.05275258
 -0.04885425  0.12481797 -0.0208328   0.03966279]


In [33]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])
# print(test_embedding_0[:200])

[-0.00578664  0.02242682 -0.01892621  0.12811586 -0.01249751 -0.07321841
 -0.00281971  0.08617394 -0.04377401  0.03073668]


## 将数据加载到 Milvus

### 创建 Collection

In [34]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。**这是为了示例代码可以重复运行**

In [35]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [36]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [37]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 416/416 [00:00<00:00, 1753598.46it/s]


{'insert_count': 416, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [45]:
# question = "How is data stored in milvus?"
question = "抵押权人实现抵押权的相关条款"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [46]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [47]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u4e8c\u8282 \u62b5\u62bc\u6743",
        0.6869308352470398
    ],
    [
        "\u7b2c\u56db\u767e\u4e8c\u5341\u4e09\u6761 \u62b5\u62bc\u6743\u4eba\u5b9e\u73b0\u62b5\u62bc\u6743\uff0c\u53ef\u4ee5\u4e0e\u62b5\u62bc\u4eba\u534f\u8bae\u4ee5\u62b5\u62bc\u8d22\u4ea7\u6298\u4ef7\u6216\u8005\u4ee5\u62cd\u5356\u3001\u53d8\u5356\u8be5\u62b5\u62bc\u8d22\u4ea7\u6240\u5f97\u7684\u4ef7\u6b3e\u4f18\u5148\u53d7\u507f\u3002\n\u534f\u8bae\u635f\u5bb3\u5176\u4ed6\u503a\u6743\u4eba\u5229\u76ca\u7684\uff0c\u5176\u4ed6\u503a\u6743\u4eba\u53ef\u4ee5\u8bf7\u6c42\u4eba\u6c11\u6cd5\u9662\u64a4\u9500\u8be5\u534f\u8bae\u3002\n\u62b5\u62bc\u6743\u4eba\u4e0e\u62b5\u62bc\u4eba\u672a\u5c31\u62b5\u62bc\u6743\u7684\u5b9e\u73b0\u65b9\u5f0f\u8fbe\u6210\u534f\u8bae\u7684\uff0c\u62b5\u62bc\u6743\u4eba\u53ef\u4ee5\u8bf7\u6c42\u4eba\u6c11\u6cd5\u9662\u62cd\u5356\u3001\u53d8\u5356\u62b5\u62bc\u8d22\u4ea7\u3002",
        0.6676101684570312
    ],
    [
        "\u7b2c\u56db\u767e\u4e09\u5341\u4e8c\u6

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [48]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [49]:
context

'第二节 抵押权\n第四百二十三条 抵押权人实现抵押权，可以与抵押人协议以抵押财产折价或者以拍卖、变卖该抵押财产所得的价款优先受偿。\n协议损害其他债权人利益的，其他债权人可以请求人民法院撤销该协议。\n抵押权人与抵押人未就抵押权的实现方式达成协议的，抵押权人可以请求人民法院拍卖、变卖抵押财产。\n第四百三十二条 最高额抵押权所担保的债权确定后，抵押权人实现抵押权，可以与抵押人协议以抵押财产折价或者以拍卖、变卖该抵押财产所得的价款优先受偿。\n协议损害其他债权人利益的，其他债权人可以请求人民法院撤销该协议。\n抵押权人与抵押人未就抵押权的实现方式达成协议的，抵押权人可以请求人民法院拍卖、变卖抵押财产。'

In [50]:
question

'抵押权人实现抵押权的相关条款'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [51]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [52]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n第二节 抵押权\n第四百二十三条 抵押权人实现抵押权，可以与抵押人协议以抵押财产折价或者以拍卖、变卖该抵押财产所得的价款优先受偿。\n协议损害其他债权人利益的，其他债权人可以请求人民法院撤销该协议。\n抵押权人与抵押人未就抵押权的实现方式达成协议的，抵押权人可以请求人民法院拍卖、变卖抵押财产。\n第四百三十二条 最高额抵押权所担保的债权确定后，抵押权人实现抵押权，可以与抵押人协议以抵押财产折价或者以拍卖、变卖该抵押财产所得的价款优先受偿。\n协议损害其他债权人利益的，其他债权人可以请求人民法院撤销该协议。\n抵押权人与抵押人未就抵押权的实现方式达成协议的，抵押权人可以请求人民法院拍卖、变卖抵押财产。\n</context>\n<question>\n抵押权人实现抵押权的相关条款\n</question>\n<translated>\n</translated>\n'

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [53]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，抵押权人实现抵押权的相关条款主要包括：

1. **协议实现方式**：抵押权人实现抵押权时，可以与抵押人协议以抵押财产折价，或者以拍卖、变卖该抵押财产所得的价款优先受偿。
2. **协议撤销权**：如果协议损害其他债权人利益，其他债权人可以请求人民法院撤销该协议。
3. **司法实现方式**：抵押权人与抵押人未就抵押权的实现方式达成协议的，抵押权人可以请求人民法院拍卖、变卖抵押财产。

这些规定适用于一般抵押权（第四百二十三条）和最高额抵押权（第四百三十二条）。

<translated>
According to the provided context, the relevant provisions for the mortgagee to realize the mortgage right mainly include:

1. **Agreement on Realization Method**: When realizing the mortgage right, the mortgagee may reach an agreement with the mortgagor to offset the mortgaged property at a discounted price or to have priority in compensation from the proceeds obtained through auction or sale of the mortgaged property.
2. **Right to Revoke the Agreement**: If the agreement harms the interests of other creditors, those creditors may request the People's Court to revoke the agreement.
3. **Judicial Realization Method**: If the mortgagee and the mortgagor fail to reach an agreement on the method of realizing the mortgage right, the mor

In [54]:
print("haha,done😁")

haha,done😁
